In [ ]:
pip install python-sat #Cài đặt thư viện python-sat

In [ ]:
pip install pypblib #Cài đặt thư viện pypblib

In [ ]:
from pysat.solvers import Glucose3
from pysat.formula import CNF
from pypblib import pblib
from pypblib.pblib import PBConfig, Pb2cnf
from typing import List, Dict, Any, Optional
import logging

In [ ]:
# Tạo lớp VariableFactory để quản lý và tạo các biến sử dụng trong bài toán RCPSP
class VariableFactory:
    def __init__(self):
        # Biến đếm số lượng biến, bắt đầu từ 1 (để mỗi biến có một ID duy nhất)
        self.var_count = 1

        # Từ điển lưu trữ ánh xạ từ tên biến (string) sang ID của biến (số nguyên)
        self.var_map = {}

    # Hàm lấy ID của một biến theo tên, nếu chưa có biến đó thì tạo mới
    def get_var(self, name):
        """
        Kiểm tra xem biến với tên `name` đã tồn tại hay chưa:
        - Nếu chưa, gán ID mới cho biến và lưu trữ nó vào `var_map`.
        - Nếu đã tồn tại, trả về ID của biến trong `var_map`.
        """
        if name not in self.var_map:
            # Gán ID mới cho biến
            self.var_map[name] = self.var_count
            # Tăng giá trị đếm số lượng biến
            self.var_count += 1
        # Trả về ID của biến
        return self.var_map[name]

    # Hàm tạo hoặc lấy biến đại diện cho trạng thái "bắt đầu" của một công việc tại một thời điểm
    def start(self, task_id, time):
        """
        Trả về ID của biến đại diện cho công việc `task_id` bắt đầu tại thời điểm `time`.
        Tên biến có định dạng: "start_task_id_time".
        """
        return self.get_var(f"start_{task_id}_{time}")

    # Hàm tạo hoặc lấy biến đại diện cho trạng thái "đang chạy" của một công việc tại một thời điểm
    def run(self, task_id, time):
        """
        Trả về ID của biến đại diện cho công việc `task_id` đang chạy tại thời điểm `time`.
        Tên biến có định dạng: "run_task_id_time".
        """
        return self.get_var(f"run_{task_id}_{time}")

    # Hàm tạo hoặc lấy biến đại diện cho trạng thái "tiêu thụ tài nguyên" của một công việc tại một thời điểm
    def consume(self, task_id, resource_id, time, index):
        """
        Trả về ID của biến đại diện cho công việc `task_id` tiêu thụ tài nguyên `resource_id`
        tại thời điểm `time` cho một phần cụ thể (với chỉ số `index`).
        Tên biến có định dạng: "consume_task_id_resource_id_time_index".
        """
        return self.get_var(f"consume_{task_id}_{resource_id}_{time}_{index}")

In [ ]:
# Equation (6): Mã hóa ràng buộc hoạt động phải bắt đầu tại một thời điểm duy nhất (ALK - AtLeastK)
def encode_unique_start_instant_alk(solver, vf, max_time, task_id, duration):
    """
    Mã hóa ràng buộc đảm bảo rằng một hoạt động chỉ có thể bắt đầu tại một thời điểm duy nhất.
    - Tạo một mệnh đề (clause) để đảm bảo rằng ít nhất một thời điểm bắt đầu được chọn.
    - Tạo thêm các mệnh đề để đảm bảo rằng không có hai thời điểm bắt đầu được chọn đồng thời.

    Parameters:
    - solver: Bộ giải SAT.
    - vf: Lớp quản lý các biến (VariableFactory).
    - max_time: Thời gian tối đa có thể bắt đầu hoạt động.
    - task_id: ID của hoạt động.
    - duration: Thời gian thực hiện hoạt động.
    """
    # Tạo mệnh đề để đảm bảo hoạt động phải bắt đầu tại ít nhất một thời điểm
    clause = [vf.start(task_id, t) for t in range(max_time - duration + 1)]
    solver.add_clause(clause)

    # Tạo các mệnh đề để đảm bảo hoạt động chỉ bắt đầu tại duy nhất một thời điểm
    for t1 in range(max_time - duration + 1):
        for t2 in range(t1 + 1, max_time - duration + 1):
            solver.add_clause([-vf.start(task_id, t1), -vf.start(task_id, t2)])

In [ ]:
# Equation (7): Mã hóa ràng buộc giới hạn thời gian bắt đầu
def encode_start_in_time(solver, vf, max_time, task_id, duration):
    """
    Mã hóa ràng buộc đảm bảo rằng hoạt động không được bắt đầu sau thời điểm kết thúc có thể.
    - Tạo các mệnh đề phủ định để loại bỏ các thời điểm không hợp lệ.

    Parameters:
    - solver: Bộ giải SAT.
    - vf: Lớp quản lý các biến (VariableFactory).
    - max_time: Thời gian tối đa.
    - task_id: ID của hoạt động.
    - duration: Thời gian thực hiện hoạt động.
    """
    # Loại bỏ các thời điểm không hợp lệ (bắt đầu sau thời điểm max_time - duration)
    for t in range(max_time - duration + 1, max_time):
        solver.add_clause([-vf.start(task_id, t)])

In [ ]:
# Equation (8) và (9): Mã hóa ràng buộc liên tục khi hoạt động đang chạy
def encode_runtime(solver, vf, max_time, task_id, duration):
    """
    Mã hóa ràng buộc đảm bảo rằng:
    - Nếu hoạt động bắt đầu tại thời điểm t, nó phải chạy liên tục trong khoảng từ t đến t + duration - 1.
    - Hoạt động không thể chạy ở thời điểm khác ngoài khoảng thời gian này.

    Parameters:
    - solver: Bộ giải SAT.
    - vf: Lớp quản lý các biến (VariableFactory).
    - max_time: Thời gian tối đa.
    - task_id: ID của công việc.
    - duration: Thời gian thực hiện công việc.
    """
    for t in range(max_time):
        start_var = vf.start(task_id, t)
        for j in range(max_time):
            run_var = vf.run(task_id, j)
            # Nếu ngoài khoảng thời gian thực hiện, không thể chạy
            if j < t or j >= t + duration:
                solver.add_clause([-start_var, -run_var])
            # Nếu trong khoảng thời gian thực hiện, phải chạy
            else:
                solver.add_clause([-start_var, run_var])

In [ ]:
# Equation (10): Mã hóa quan hệ "Finish-to-Start"
def encode_relation_fs(solver, vf, max_time, task1, task2, duration1):
    """
    Mã hóa ràng buộc "Finish-to-Start" giữa hai công việc.
    - Nếu công việc 1 kết thúc tại thời điểm t, công việc 2 không thể bắt đầu trước thời điểm t + duration1.

    Parameters:
    - solver: Bộ giải SAT.
    - vf: Lớp quản lý các biến (VariableFactory).
    - max_time: Thời gian tối đa.
    - task1: ID của công việc 1.
    - task2: ID của công việc 2.
    - duration1: Thời gian thực hiện công việc 1.
    """
    for t in range(max_time):
        start_var = vf.start(task1, t)
        for k in range(t + duration1):  # Changed this part
            solver.add_clause([-start_var, -vf.start(task2, k)])

In [ ]:
# Equation (16): Mã hóa atoms tiêu thụ tài nguyên
def encode_consumption_atoms(solver, vf, max_time, tasks, resources):
    """
    Mã hóa ràng buộc tiêu thụ tài nguyên của các hoạt động:
    - Nếu một công việc đang chạy tại thời điểm t, nó phải tiêu thụ một lượng tài nguyên nhất định.

    Parameters:
    - solver: Bộ giải SAT.
    - vf: Lớp quản lý các biến (VariableFactory).
    - max_time: Thời gian tối đa.
    - tasks: Danh sách các hoạt động.
    - resources: Danh sách các tài nguyên.
    """
    for t in range(max_time):
        for task in tasks:
            for resource in resources:
                # Lấy số lượng tài nguyên tiêu thụ
                consumption = task.get("consumption", {}).get(resource["id"], 0)
                for i in range(consumption):
                    # Nếu công việc đang chạy, tài nguyên phải được tiêu thụ
                    solver.add_clause([-vf.run(task["id"], t), vf.consume(task["id"], resource["id"], t, i)])

In [ ]:
# Equation (5 and 17): BCC resource constraint
def encode_resource_constraint_cardinality(solver, vf, max_time, tasks, resources):
    """
    Mã hóa ràng buộc tài nguyên bằng BCC sử dụng Sequential Counter (NSC) với PBLib.

    Parameters:
    - solver: Bộ giải SAT.
    - vf: Lớp quản lý các biến (VariableFactory).
    - max_time: Thời gian tối đa.
    - tasks: Danh sách các hoạt động.
    - resources: Danh sách các tài nguyên.
    """
    id_variable = int  # Biến toàn cục để đếm số lượng biến được tạo ra trong quá trình mã hóa

    pbConfig = PBConfig()  # Cấu hình cho PBLib
    pbConfig.set_AMK_Encoder(pblib.AMK_CARD)  # Dùng NSC Encoder cho AtMostK (NSC là một thuật toán mã hóa cho BCC)
    pb2 = Pb2cnf(pbConfig)  # Tạo đối tượng Pb2cnf để mã hóa các ràng buộc

    # Lặp qua từng thời điểm từ 0 đến max_time
    for t in range(max_time):
        # Lặp qua từng tài nguyên
        for resource in resources:
            # Lấy các biến tiêu thụ tài nguyên cho từng công việc tại thời điểm t
            consumption_vars = [
                vf.consume(task["id"], resource["id"], t, i)  # Biến tiêu thụ tài nguyên của công việc tại thời điểm t
                for task in tasks
                for i in range(task.get("consumption", {}).get(resource["id"], 0))  # Lấy số lượng tài nguyên công việc tiêu thụ
            ]

            # Nếu có các biến tiêu thụ tài nguyên (tức là có công việc tiêu thụ tài nguyên tại thời điểm t)
            if consumption_vars:
                # Khởi tạo công thức cho BCC
                formula = []
                weights = [1] * len(consumption_vars)  # Tạo trọng số cho các biến (tất cả trọng số là 1)

                # Mã hóa ràng buộc "AtMostK" với NSC: đảm bảo rằng không vượt quá dung lượng tài nguyên
                max_var = pb2.encode_at_most_k(
                    weights, consumption_vars, resource["capacity"], formula, id_variable
                )

                # Cập nhật id_variable sau khi mã hóa
                id_variable = max_var + 1

                # Thêm các ràng buộc (clauses) vào solver
                for clause in formula:
                    solver.add_clause(clause)

In [ ]:
# Khởi tạo bài toán đầu vào
def parse_input():
    tasks = [
        {"id": 0, "duration": 308, "name": "Task 0"},
        {"id": 1, "duration": 10, "name": "Task 1"},
        {"id": 2, "duration": 1, "name": "Task 2"},
        {"id": 3, "duration": 20, "name": "Task 3"},
        {"id": 4, "duration": 2, "name": "Task 4"},
        {"id": 5, "duration": 5, "name": "Task 5"},
        {"id": 6, "duration": 1, "name": "Task 6"}
    ]

    # relations (aob, ea)
    relations = [
        {"task_id_1": 1, "task_id_2": 2, "relation_type": "ea"},
        {"task_id_1": 2, "task_id_2": 3, "relation_type": "ea"},
        {"task_id_1": 3, "task_id_2": 4, "relation_type": "ea"},
        {"task_id_1": 4, "task_id_2": 5, "relation_type": "ea"},
        {"task_id_1": 5, "task_id_2": 6, "relation_type": "ea"}
    ]

    # consumption data (task_id, resource_id, amount)
    consumptions = [
        {"task_id": 1, "resource_id": 0, "amount": -3},
        {"task_id": 3, "resource_id": 0, "amount": -3},
        {"task_id": 3, "resource_id": 1, "amount": -3},
        {"task_id": 3, "resource_id": 2, "amount": -3},
        {"task_id": 4, "resource_id": 0, "amount": -3},
        {"task_id": 4, "resource_id": 3, "amount": -3},
        {"task_id": 4, "resource_id": 4, "amount": -3},
        {"task_id": 5, "resource_id": 0, "amount": -3},
        {"task_id": 5, "resource_id": 3, "amount": -3},
        {"task_id": 5, "resource_id": 4, "amount": -3}
    ]

    # resources (capacity and names)
    resources = [
        {"id": 0, "capacity": 8, "name": "Resource 0"},
        {"id": 1, "capacity": 8, "name": "Resource 1"},
        {"id": 2, "capacity": 8, "name": "Resource 2"},
        {"id": 3, "capacity": 8, "name": "Resource 3"},
        {"id": 4, "capacity": 8, "name": "Resource 4"}
    ]

    return tasks, relations, consumptions, resources

In [ ]:
def validate_input_data(tasks: List[Dict], relations: List[Dict],
                         consumptions: List[Dict], resources: List[Dict]) -> bool:
    """
    Validate the input data before solving the RCPSP problem

    :param tasks: List of task dictionaries
    :param relations: List of precedence relations
    :param consumptions: List of resource consumption details
    :param resources: List of resource details
    :return: True if input is valid, False otherwise
    """
    # Configure logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s - %(levelname)s: %(message)s')
    logger = logging.getLogger(__name__)

    validation_passed = True

    # 1. Task Validation
    if not tasks:
        logger.error("No tasks defined")
        validation_passed = False

    for task in tasks:
        # Check task duration is positive
        if task.get('duration', 0) <= 0:
            logger.error(f"Invalid duration for task {task.get('id')}: {task.get('duration')}")
            validation_passed = False

    # 2. Precedence Relations Validation
    task_ids = {task['id'] for task in tasks}
    for relation in relations:
        if relation['task_id_1'] not in task_ids:
            logger.error(f"Invalid first task in relation: {relation['task_id_1']}")
            validation_passed = False
        if relation['task_id_2'] not in task_ids:
            logger.error(f"Invalid second task in relation: {relation['task_id_2']}")
            validation_passed = False

    # 3. Resource Validation
    for resource in resources:
        if resource.get('capacity', 0) <= 0:
            logger.error(f"Invalid resource capacity for resource {resource.get('id')}")
            validation_passed = False

    # 4. Consumption Validation
    consumption_task_ids = {cons['task_id'] for cons in consumptions}
    if not consumption_task_ids.issubset(task_ids):
        invalid_tasks = consumption_task_ids - task_ids
        logger.error(f"Consumption defined for non-existent tasks: {invalid_tasks}")
        validation_passed = False

    return validation_passed

In [ ]:
def decode_solution(tasks, model, vf):
    """
    Decode the SAT solver's model to extract task start times and schedule details.

    :param tasks: List of task dictionaries
    :param model: Raw model from the SAT solver
    :param vf: VariableFactory used in encoding
    :return: Decoded schedule information
    """
    # Create reverse mapping for variables
    reverse_var_map = {v: k for k, v in vf.var_map.items()}

    # Extract positive variables from the model
    positive_vars = set(abs(var) for var in model if var > 0)

    # Store start times for tasks
    task_start_times = {}

    # Decode start times
    for task in tasks:
        task_start_found = False
        for time in range(len(reverse_var_map)):
            start_var = vf.start(task['id'], time)
            if start_var in positive_vars:
                task_start_times[task['id']] = time
                task_start_found = True
                break

        if not task_start_found:
            print(f"Warning: No start time found for task {task['id']}")

    # Sort tasks by start time
    sorted_tasks = sorted(tasks, key=lambda x: task_start_times.get(x['id'], float('inf')))

    # Prepare detailed schedule
    schedule = []
    for task in sorted_tasks:
        start_time = task_start_times.get(task['id'])
        if start_time is not None:
            end_time = start_time + task['duration']

            # Collect resource consumption
            task_resources = []
            for consumption in consumptions:
                if consumption['task_id'] == task['id']:
                    task_resources.append({
                        'resource_id': consumption['resource_id'],
                        'amount': consumption['amount']
                    })

            schedule.append({
                'task_id': task['id'],
                'task_name': task['name'],
                'start_time': start_time,
                'end_time': end_time,
                'duration': task['duration'],
                'resources_consumed': task_resources
            })

    return schedule

In [ ]:
def print_schedule(schedule):
    """
    Print the decoded schedule in a human-readable format.

    :param schedule: Decoded schedule from decode_solution
    """

    print("{:<10} {:<15} {:<12} {:<12} {:<15}".format(
        "Task ID", "Task Name", "Start Time", "End Time", "Resources Consumed"
    ))
    print("-" * 70)

    for task in schedule:
        # Format resource consumption string
        resource_str = ", ".join([
            f"R{r['resource_id']}: {r['amount']}" for r in task['resources_consumed']
        ]) if task['resources_consumed'] else "None"

        print("{:<10} {:<15} {:<12} {:<12} {:<15}".format(
            task['task_id'],
            task['task_name'],
            task['start_time'],
            task['end_time'],
            resource_str
        ))

In [ ]:
def validate_schedule(schedule, tasks, relations, consumptions, resources, max_time):
    """
    Validates RCPSP solution schedule.

    Param:
        schedule (list): List các tác vụ đã lên lịch với thời gian bắt đầu/kết thúc và mức sử dụng tài nguyên
        tasks (list): List các tác vụ đầu vào
        relations (list): Mối quan hệ ưu tiên giữa các nhiệm vụ
        consumptions (list): Mức tài nguyên tiêu thụ cho từng nhiệm vụ
        resources (list): List tài nguyên
        max_time (int): Thời gian tối đa

    Returns:
        tuple: (bool, dict) - (is_valid, validation_results)
    """
    validation_results = {
        'task_coverage': {'passed': True, 'details': []},
        'time_windows': {'passed': True, 'details': []},
        'precedence': {'passed': True, 'details': []},
        'resources': {'passed': True, 'details': []}
    }

    # 1. Check if all tasks are scheduled exactly once
    scheduled_task_ids = {task['task_id'] for task in schedule}
    original_task_ids = {task['id'] for task in tasks}

    if scheduled_task_ids != original_task_ids:
        validation_results['task_coverage']['passed'] = False
        missing_tasks = original_task_ids - scheduled_task_ids
        extra_tasks = scheduled_task_ids - original_task_ids
        if missing_tasks:
            validation_results['task_coverage']['details'].append(
                f"Missing tasks in schedule: {missing_tasks}")
        if extra_tasks:
            validation_results['task_coverage']['details'].append(
                f"Extra tasks in schedule: {extra_tasks}")
    else:
        validation_results['task_coverage']['details'].append(
            "All tasks are scheduled exactly once")

    # 2. Check if all tasks are within the maximum time window
    for task in schedule:
        if task['start_time'] < 0:
            validation_results['time_windows']['passed'] = False
            validation_results['time_windows']['details'].append(
                f"Task {task['task_id']} starts before time 0")
        if task['end_time'] > max_time:
            validation_results['time_windows']['passed'] = False
            validation_results['time_windows']['details'].append(
                f"Task {task['task_id']} ends after max time {max_time}")

        # Verify task duration
        original_duration = next(t['duration'] for t in tasks if t['id'] == task['task_id'])
        if task['end_time'] - task['start_time'] != original_duration:
            validation_results['time_windows']['passed'] = False
            validation_results['time_windows']['details'].append(
                f"Task {task['task_id']} duration mismatch: "
                f"scheduled {task['end_time'] - task['start_time']}, "
                f"required {original_duration}")

    if validation_results['time_windows']['passed']:
        validation_results['time_windows']['details'].append(
            "All tasks are within their allowed time windows and have correct durations")

    # 3. Check precedence relations (Finish-to-Start)
    task_times = {task['task_id']: (task['start_time'], task['end_time'])
                 for task in schedule}

    precedence_checked = 0
    for relation in relations:
        precedence_checked += 1
        pred_task = relation['task_id_1']
        succ_task = relation['task_id_2']
        if task_times[pred_task][1] > task_times[succ_task][0]:
            validation_results['precedence']['passed'] = False
            validation_results['precedence']['details'].append(
                f"Precedence violation: Task {pred_task} must finish before "
                f"Task {succ_task} starts")

    if validation_results['precedence']['passed']:
        validation_results['precedence']['details'].append(
            f"All {precedence_checked} precedence relations are satisfied")

    # 4. Check resource constraints
    resource_usage = {res['id']: [0] * max_time for res in resources}

    # Calculate resource usage over time
    for task in schedule:
        task_resources = task.get('resources_consumed', [])
        for res in task_resources:
            resource_id = res['resource_id']
            amount = abs(res['amount'])
            for t in range(task['start_time'], task['end_time']):
                resource_usage[resource_id][t] += amount

    # Check if resource usage exceeds capacity
    resources_checked = 0
    for resource in resources:
        resources_checked += 1
        res_id = resource['id']
        capacity = resource['capacity']
        max_usage = max(resource_usage[res_id])

        if max_usage > capacity:
            validation_results['resources']['passed'] = False
            for t, usage in enumerate(resource_usage[res_id]):
                if usage > capacity:
                    validation_results['resources']['details'].append(
                        f"Resource {res_id} overused at time {t}: "
                        f"usage {usage} > capacity {capacity}")
        else:
            validation_results['resources']['details'].append(
                f"Resource {res_id} usage is within capacity (max usage: {max_usage}/{capacity})")

    if validation_results['resources']['passed']:
        validation_results['resources']['details'].insert(
            0, f"All {resources_checked} resources are within their capacity limits")

    # Overall validation result
    is_valid = all(result['passed'] for result in validation_results.values())
    return is_valid, validation_results

def print_validation_result(is_valid, validation_results):
    """
    Prints the validation results in a detailed, formatted way.
    """
    print("\nRCPSP Schedule Validation Report")
    print("================================")

    # Function to print section results
    def print_section(title, results):
        print(f"\n{title}")
        print("-" * len(title))
        if results['passed']:
            print("✓ PASSED")
        else:
            print("✗ FAILED")
        for detail in results['details']:
            print(f"  • {detail}")

    # Print each validation section
    print_section("Task Coverage Check", validation_results['task_coverage'])
    print_section("Time Windows Check", validation_results['time_windows'])
    print_section("Precedence Relations Check", validation_results['precedence'])
    print_section("Resource Capacity Check", validation_results['resources'])

    # Print overall result
    print("\nOverall Validation Result")
    print("=======================")
    if is_valid:
        print("✓ PASSED - Tất cả các ràng buộc đều được thỏa mãn")
    else:
        print("✗ FAILED - Một số ràng buộc bị vi phạm")

In [ ]:
def solve_rcpsp(max_time, tasks, relations, consumptions, resources):
    solver = Glucose3()
    vf = VariableFactory()

    # Encoding tasks with ALK, start time, runtime constraints
    for task in tasks:
        encode_unique_start_instant_alk(solver, vf, max_time, task["id"], task["duration"])
        encode_start_in_time(solver, vf, max_time, task["id"], task["duration"])
        encode_runtime(solver, vf, max_time, task["id"], task["duration"])

    # Encoding precedence relations (Finish-to-Start)
    for relation in relations:
        task_1 = relation["task_id_1"]
        task_2 = relation["task_id_2"]
        task_1_duration = next(t["duration"] for t in tasks if t["id"] == task_1)
        encode_relation_fs(solver, vf, max_time, task_1, task_2, task_1_duration)

    # Encoding resource consumption
    for task in tasks:
        task.setdefault("consumption", {})

    for consumption in consumptions:
        task = next(t for t in tasks if t["id"] == consumption["task_id"])
        task["consumption"][consumption["resource_id"]] = consumption["amount"]

    # Encoding resource constraints
    encode_resource_constraint_cardinality(solver, vf, max_time, tasks, resources)

    # Solve the problem
    if solver.solve():
        model = solver.get_model()
        print("SAT: Có lời giải.")

        # Decode and print the solution
        decoded_schedule = decode_solution(tasks, model, vf)
        print_schedule(decoded_schedule)

        solver.delete()
        return model, vf
    else:
        print("UNSAT: Không có lời giải.")
        solver.delete()
        return None, None

In [ ]:
tasks, relations, consumptions, resources = parse_input()
if validate_input_data(tasks, relations, consumptions, resources):
      print("Dữ liệu đầu vào hợp lệ.")
else:
      print("Dữ liệu đầu vào không hợp lệ.")

max_time = 350
model, vf = solve_rcpsp(max_time, tasks, relations, consumptions, resources)

if model is not None:
    decoded_schedule = decode_solution(tasks, model, vf)
    is_valid, validation_errors = validate_schedule(
        decoded_schedule, tasks, relations, consumptions, resources, max_time
    )
    print_validation_result(is_valid, validation_errors)

Dữ liệu đầu vào hợp lệ.
SAT: Có lời giải.
Task ID    Task Name       Start Time   End Time     Resources Consumed
----------------------------------------------------------------------
0          Task 0          29           337          None           
1          Task 1          45           55           R0: -3         
2          Task 2          58           59           None           
3          Task 3          150          170          R0: -3, R1: -3, R2: -3
4          Task 4          170          172          R0: -3, R3: -3, R4: -3
5          Task 5          227          232          R0: -3, R3: -3, R4: -3
6          Task 6          233          234          None           

RCPSP Schedule Validation Report

Task Coverage Check
-------------------
✓ PASSED
  • All tasks are scheduled exactly once

Time Windows Check
------------------
✓ PASSED
  • All tasks are within their allowed time windows and have correct durations

Precedence Relations Check
--------------------------
✓ PA